# Calibration lamp

In [6]:
%matplotlib widget
from core.experiment import *
from scipy.interpolate import interp1d

## Load the data from folder

In [7]:
spectrometers = ["2101182U1","2101183U1","2101184U1","2101207U1"]

for i in range(0,len(spectrometers)):
    subplots()
    print(spectrometers[i])
    spec_l = spectrometers[i]
    folder_cal = "D:\\Data_LIBS\\4 channel Spectrometer\\AvaLight-DH-CAL\\Avantes CAL\\"+spec_l+"\\"
    folder_cal_dark = "D:\\Data_LIBS\\4 channel Spectrometer\\AvaLight-DH-CAL\\Avantes CAL - Dark\\"+spec_l+"\\"

    filenames_cal = [d for d in os.listdir(folder_cal) if d.endswith(".TXT") or d.endswith(".txt")]
    filenames_dark = [d for d in os.listdir(folder_cal_dark) if d.endswith(".TXT") or d.endswith(".txt")]
    
    
    sg_dark = signal_spec(folder_cal,filenames_dark[0],start_empty = True)
    sg_dark.from_data(read_data(folder_cal_dark+filenames_dark[0]),folder_cal)

    i_cal_waves = []
    i_cal_ints = []
    current_exp = experiment(folder_cal)
    sg = current_exp.mean_signal #signal_spec(folder_cal,filenames_cal[0], start_empty=True)
    sg.from_data(read_data(folder_cal+filenames_cal[0]),folder_cal)
    i_cal_waves = sg.wavelengths.flatten()+0.21
    i_cal_ints = (sg.spectrum.flatten()-sg_dark.spectrum.flatten())/sg.integration_time




    print(sg.integration_time)
    print(sg.spectrometer_labels)



    #calibration files paths
    cal_path_halogen = 'D:\\Data_LIBS\\ficheiros calibracao\\2101007-Halogen CC-VISNIR.lmp'
    cal_path_deuterium = 'D:\\Data_LIBS\\ficheiros calibracao\\2101007-Deuterium-Halogen CC-VISNIR.lmp'

    ############################################
    #load data
    cal_data_halogen = np.loadtxt(cal_path_halogen,dtype=str)
    cal_data_halogen[:,0]
    cal_waves_halogen = []
    cal_ints_halogen = [] 
    for i in range(0,len(cal_data_halogen[:,1])):
        cal_waves_halogen.append(float(cal_data_halogen[i,0].replace(',','.')))
        cal_ints_halogen.append(float(cal_data_halogen[i,1].replace(',','.')))

    cal_data_deuterium = np.loadtxt(cal_path_deuterium ,dtype=str)
    cal_data_deuterium [:,0]
    cal_waves_deuterium  = []
    cal_ints_deuterium  = [] 
    for i in range(0,len(cal_data_deuterium[:,1])):
        cal_waves_deuterium.append(float(cal_data_deuterium[i,0].replace(',','.')))
        cal_ints_deuterium.append(float(cal_data_deuterium[i,1].replace(',','.')))
    #############################################

    #make interpolator functions
    kk = 'nearest'

    f_halogen = interp1d(cal_waves_halogen, cal_ints_halogen, kind=kk,fill_value='extrapolate')
    f_deuterium = interp1d(cal_waves_deuterium, cal_ints_deuterium, kind=kk,fill_value='extrapolate')

    x_hal = np.linspace(cal_waves_halogen[0],cal_waves_halogen[-1],1000) 
    x_deut = np.linspace(cal_waves_deuterium[0],cal_waves_deuterium[-1],1000)

    subplot(311)
    plot(cal_waves_halogen, cal_ints_halogen,'o',ms=1,label = 'Halogen')
    plot(x_hal, f_halogen(x_hal),'-',lw=0.5,label = 'Halogen - interpolator')
    plot(x_hal, f_halogen(x_hal)+f_deuterium(x_hal),'-',lw=0.5,label = 'Sum')

    plot(cal_waves_deuterium, cal_ints_deuterium,'x',ms=1,label = 'Deuterium')
    plot(x_deut, f_deuterium(x_deut),'-',lw=0.5,label = 'Deuterium - interpolator')

    legend()

    init = 20
    end = -20
    subplot(312)
    title('Calibration Factor')
    plot(i_cal_waves[init:end],i_cal_ints[init:end],'-',lw=0.5)
    print(i_cal_waves)
    plot(i_cal_waves[init:end],(f_halogen(i_cal_waves[init:end])+f_deuterium(i_cal_waves[init:end])),'-',lw=0.5,label = 'Sum')

    filter_d = i_cal_ints > 0.2

    np.savetxt(folder_cal+"cal_factor.cal",
               ((f_halogen(i_cal_waves)+f_deuterium(i_cal_waves))/i_cal_ints)*filter_d)
    
    




    def calibration_factor(spectrometer_label):
        try:
            data_cal = np.loadtxt("D:\\Data_LIBS\\4 channel Spectrometer\\AvaLight-DH-CAL\\Avantes CAL\\"+spectrometer_label+"\\cal_factor.cal")
            return signal.savgol_filter(data_cal, 21, 3) 
        except:
            print("Calibration File not found")
            return 1

    data_cal = calibration_factor(spec_l)
    subplot(313)
    plot(i_cal_waves[init:end],(f_halogen(i_cal_waves[init:end])+f_deuterium(i_cal_waves[init:end]))/i_cal_ints[init:end],'-',lw=0.5)

    plot(i_cal_waves,data_cal,'-',lw=0.5)







Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2101182U1
5000.0
['o']
[479.67 479.75 479.83 ... 613.94 613.99 614.04]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2101183U1
10000.0
['o']
[331.19 331.27 331.36 ... 488.85 488.91 488.97]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2101184U1
10000.0
['o']
[217.71 217.78 217.84 ... 339.78 339.83 339.88]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2101207U1
200.0
['o']
[607.55 607.67 607.8  ... 826.86 826.95 827.04]


In [3]:
#subplots()

folder_cal = "D:\\Data_LIBS\\Avantes CAL\\"

folder_cal_dark = "D:\\Data_LIBS\\Avantes CAL - Dark\\"
filenames_cal = [d for d in os.listdir(folder_halogen) if d.endswith(".TXT") or d.endswith(".txt")]

filenames_halogen = [d for d in os.listdir(folder_halogen) if d.endswith(".TXT") or d.endswith(".txt")]
i_cal_waves_deuterium = []
i_cal_ints_deuterium = []

i_cal_waves_halogen = []
i_cal_ints_halogen = []

limit_r = 100
limit_l = 100

for i in range(0,len(filenames_halogen)):
    print(filenames_halogen[i])
    sg = signal_spec(folder_halogen,filenames_halogen[i], start_empty=True)
    sg.from_data(read_data(folder+filenames_halogen[i]),folder)
    
    sg_dark = signal_spec(folder_halogen,filenames_halogen_dark[i], start_empty=True)
    
    i_cal_waves_halogen.append(sg.wavelengths[limit_l:-limit_r].flatten())
    print(sg.integration_time)
    print(sg.spectrometer_labels)
    i_cal_ints_halogen.append(sg.spectrum[limit_l:-limit_r].flatten()/sg.integration_time)
    #plot(sg.wavelengths,sg.spectrum,'-',label=filenames[i].split('_')[0])

i_cal_waves_halogen0 = np.array(i_cal_waves_halogen).flatten()
i_cal_ints_halogen = np.array(i_cal_ints_halogen).flatten()
    
#print(sorted(zip(i_cal_waves_halogen,i_cal_ints_halogen), key = lambda x: x[0]))
i_cal_waves_halogen = [x for x,y in sorted(zip(i_cal_waves_halogen0,i_cal_ints_halogen))]
i_cal_ints_halogen = [y for x,y in sorted(zip(i_cal_waves_halogen0,i_cal_ints_halogen))]
subplots()
plot(i_cal_waves_halogen,i_cal_ints_halogen)
#print(i_cal_waves_halogen)
#f_deuterium = interp1d(i_cal_waves_deuterium, i_cal_ints_deuterium, kind='cubic')
#f_halogen = interp1d(i_cal_waves_halogen, i_cal_ints_halogen, kind='cubic')

NameError: name 'folder_halogen' is not defined

In [ ]:
sg.reference

In [ ]:


#calibration files paths
cal_path_halogen = 'D:\\Data_LIBS\\ficheiros calibracao\\2101007-Halogen CC-VISNIR.lmp'
cal_path_deuterium = 'D:\\Data_LIBS\\ficheiros calibracao\\2101007-Deuterium-Halogen CC-VISNIR.lmp'

############################################
#load data
cal_data_halogen = np.loadtxt(cal_path_halogen,dtype=str)
cal_data_halogen[:,0]
cal_waves_halogen = []
cal_ints_halogen = [] 
for i in range(0,len(cal_data_halogen[:,1])):
    cal_waves_halogen.append(float(cal_data_halogen[i,0].replace(',','.')))
    cal_ints_halogen.append(float(cal_data_halogen[i,1].replace(',','.')))

cal_data_deuterium = np.loadtxt(cal_path_deuterium ,dtype=str)
cal_data_deuterium [:,0]
cal_waves_deuterium  = []
cal_ints_deuterium  = [] 
for i in range(0,len(cal_data_deuterium[:,1])):
    cal_waves_deuterium.append(float(cal_data_deuterium[i,0].replace(',','.')))
    cal_ints_deuterium.append(float(cal_data_deuterium[i,1].replace(',','.')))
#############################################

#make interpolator functions
f_halogen = interp1d(cal_waves_halogen, cal_ints_halogen, kind='cubic')
f_deuterium = interp1d(cal_waves_deuterium, cal_ints_deuterium, kind='cubic')

x_hal = np.linspace(cal_waves_halogen[0],cal_waves_halogen[-1],1000) 
x_deut = np.linspace(cal_waves_deuterium[0],cal_waves_deuterium[-1],1000)

subplots()
plot(cal_waves_halogen, cal_ints_halogen,'o',ms=1,label = 'Halogen')
plot(x_hal, f_halogen(x_hal),'-',lw=0.5,label = 'Halogen - interpolator')
plot(x_hal, f_halogen(x_hal)+f_deuterium(x_hal),'-',lw=0.5,label = 'Sum')

plot(cal_waves_deuterium, cal_ints_deuterium,'x',ms=1,label = 'Deuterium')
plot(x_deut, f_deuterium(x_deut),'-',lw=0.5,label = 'Deuterium - interpolator')

legend()



In [4]:
def calibration_factor_halogen(wl):
    return f_halogen(wl)/i_cal_halogen(wl)

def calibration_factor_deuterium(wl):
    return f_deuterium(wl)/i_cal_deuterium(wl)

In [5]:
sg.wavelengths

array([607.34, 607.46, 607.59, ..., 826.65, 826.74, 826.83])

# Varying the number of shots

In [ ]:
current_experiment.list_of_signals

In [ ]:

intensities=[]
for i in range(0, len(current_experiment.list_of_signals)):
    ratio_of_maximum = 0.5
    wavelengths = np.concatenate([w for w in current_experiment.list_of_signals[i].wavelengths]) 
    spectrum = np.concatenate([s for s in current_experiment.list_of_signals[i].spectrum ])
    radius = .2
    ritz = 302
    intensity = get_peak_area(ritz,ratio_of_maximum , wavelengths, spectrum, radius, False)
    intensities.append(intensity)

subplots()
plot(intensities)

# Computing density
Using the H_alpha line from the Balmer series.

In [ ]:
density, x,y,p = sg.density_H_alpha()
print(str(density) + " cm^-3")
density= density*(10**2)**3

# Remove Baseline

In [ ]:
sg.remove_baseline()

# Computing the plasma temperature
Using the Saha-Boltzmann method and choosing the lines

In [ ]:
#Choose the element for computing the relevant lines
al = element('Fe')

Tp = 1.*T_ref

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 1
ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

ion_energies = al.ion_energies

#ion state 1
nlines = 5
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 2
nlines = 2
l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],al,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)


# LTE conditions
Probing the Mcwrither criterium

In [ ]:
minimum_density = sg.LTE_MW_criterium(4.8,temperature)
print("Minimum density in cm^-3: " + str("%e" %minimum_density))
print("Computed density in cm^-3: " + str("%e" %(density*(10**-2)**3)))
print("MW criterion - " + str(density>(minimum_density*(10**2)**3)))

In [ ]:
#construct a digital twin from the table data
current = concentration_data.loc[sample_f]
element_names = concentration_data.columns.values[1:]
vals =  current.values[1:]
sample_elements = []
for i in range(0,len(vals)):
    #sample_elements.append([element_names[i],vals[i]/vals[-1]])
    sample_elements.append([element_names[i],vals[i]])


sample = digital_twin(sample_elements)
for i in range(0,len(sample.list_of_elements)):
    sample.list_of_elements[i].ratio = sample.list_of_elements[i].ratio/(sample.list_of_elements[i].mass/ua)

print(sample)

for n_spectrometer in range(0,len(sg.wavelengths)):
    ll = sg.wavelengths[n_spectrometer][0]
    ul = sg.wavelengths[n_spectrometer][-1]

    #ion state 1
    nlines = 3
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, 
                                                        n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #line to normalize the intensity for direct
    line_norm1=lines[1]
    print("# Spectrometer 1 normalized to " + str(line_norm1))

    sg.compare_to_digital_sample(sample, spectrometer=n_spectrometer,max_ion_state=2, 
                                 electron_temperature = temperature,electron_density=density,
                                 d_lambda=0.02, use_wavelengths=True,line_normalize=line_norm1,Plotline = True, resolution = 5000)

# Peak detection and comparison

In [ ]:
n_spectrometer = 3

ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 3
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, 
                                                        n_lines = nlines,lower_limit = ll, upper_limit = ul)

#line to normalize the intensity for direct
line_norm1=lines[1]
print("# Spectrometer 1 normalized to " + str(line_norm1))
sg.wavelengths[n_spectrometer] = sg.wavelengths[n_spectrometer]+0.05

sg.compare_and_peaks(sample, spectrometer=n_spectrometer,max_ion_state=2, 
                                 electron_temperature = temperature,electron_density=density,
                                 d_lambda=0.02, use_wavelengths=True,line_normalize=line_norm1,Plotline = True, resolution = 5000)

# Map of lines

In [ ]:
Tp = temperature
sample1 = digital_twin([['Fe',1]])
specs_0=[]
for i in range(0,len(sg.wavelengths)):
    wl,spec,label, n_ion, specs = sample1.spectrum_NIST(wl = sg.wavelengths[i], electron_temperature=Tp, electron_density = density, max_ion_state=2,resolution=10000)
    specs_0.append(specs)

In [ ]:
specs_0
for i in range(0,len(specs_0)):
    subplots()
    subplot(311)
    plot(sg.spectrum[i])
    subplot(312)
    plot(specs_0[i][0])
    subplot(313)
    conv = np.convolve(sg.spectrum[i],specs_0[i][0][::-1], mode='full')
    #conv = np.convolve(specs_0[i][0],specs_0[i][0])
    plot(np.arange(-len(sg.spectrum[i])+1,len(sg.spectrum[i])),log10(conv))


In [ ]:
print(sg.spectrum[0].shape)
specs_0[0][0].shape

# Map of Lines in each spectrometer

In [ ]:
Tp = temperature
for i in range(0,len(sg.wavelengths)):
    wl,spec,label, n_ion, specs = sample.spectrum_NIST(wl = sg.wavelengths[i], electron_temperature=Tp, electron_density = density, max_ion_state=3,d_lambda=0.01, Map=True)
    ax = gca()
    ax.set_title("Map of lines in Spectrometer "+ str(i))

#

In [ ]:
#Choose the element for computing the relevant lines
al = element('P')

Tp = 1.*T_ref

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 6
ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 2
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 2
nlines = 2
l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

lines2 = delete(lines2, 0)
#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],al,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)







In [ ]:
lines2

In [ ]:
sample = digital_twin([['Li',1],['Pb',1],['Zn',1],['Cu',1]])

Tp = temperature
for i in range(0,len(sg.wavelengths)):
    wl,spec,label, n_ion, specs = sample.spectrum_NIST( lower_limit = 250, upper_limit = 800, max_ion_state=3,d_lambda=0.01, Map=True)
    ax = gca()
    ax.set_title("Map of lines in Spectrometer "+ str(i))

In [ ]:
from scipy import interpolate
i=2
tck = interpolate.splrep(sg.wavelengths[i],sg.spectrum[i])
xnew = np.linspace(sg.wavelengths[i][0], sg.wavelengths[i][-1], 5000)
ynew = interpolate.splev(xnew, tck, der=0)
subplots()
plot(xnew,ynew/max(ynew))
plot(sg.wavelengths[i],sg.spectrum[i]/max(sg.spectrum[i]), 'o', markersize=1,color='k')
#plot(sg.wavelengths[i],specs_0[i][0]/(200))
g=gradient(ynew,xnew)
gg=gradient(g,xnew)
plot(xnew,g/max(abs(g)),ls='-')
plot(xnew,gg/max(abs(gg)),ls='-')

In [ ]:
import numpy as np
import time
import math


def tricubic(x):
    y = np.zeros_like(x)
    idx = (x >= -1) & (x <= 1)
    y[idx] = np.power(1.0 - np.power(np.abs(x[idx]), 3), 3)
    return y


class Loess(object):

    @staticmethod
    def normalize_array(array):
        min_val = np.min(array)
        max_val = np.max(array)
        return (array - min_val) / (max_val - min_val), min_val, max_val

    def __init__(self, xx, yy, degree=1):
        self.n_xx, self.min_xx, self.max_xx = self.normalize_array(xx)
        self.n_yy, self.min_yy, self.max_yy = self.normalize_array(yy)
        self.degree = degree

    @staticmethod
    def get_min_range(distances, window):
        min_idx = np.argmin(distances)
        n = len(distances)
        if min_idx == 0:
            return np.arange(0, window)
        if min_idx == n-1:
            return np.arange(n - window, n)

        min_range = [min_idx]
        while len(min_range) < window:
            i0 = min_range[0]
            i1 = min_range[-1]
            if i0 == 0:
                min_range.append(i1 + 1)
            elif i1 == n-1:
                min_range.insert(0, i0 - 1)
            elif distances[i0-1] < distances[i1+1]:
                min_range.insert(0, i0 - 1)
            else:
                min_range.append(i1 + 1)
        return np.array(min_range)

    @staticmethod
    def get_weights(distances, min_range):
        max_distance = np.max(distances[min_range])
        weights = tricubic(distances[min_range] / max_distance)
        return weights

    def normalize_x(self, value):
        return (value - self.min_xx) / (self.max_xx - self.min_xx)

    def denormalize_y(self, value):
        return value * (self.max_yy - self.min_yy) + self.min_yy

    def estimate(self, x, window, use_matrix=False, degree=1):
        n_x = self.normalize_x(x)
        distances = np.abs(self.n_xx - n_x)
        min_range = self.get_min_range(distances, window)
        weights = self.get_weights(distances, min_range)

        if use_matrix or degree > 1:
            wm = np.multiply(np.eye(window), weights)
            xm = np.ones((window, degree + 1))

            xp = np.array([[math.pow(n_x, p)] for p in range(degree + 1)])
            for i in range(1, degree + 1):
                xm[:, i] = np.power(self.n_xx[min_range], i)

            ym = self.n_yy[min_range]
            xmt_wm = np.transpose(xm) @ wm
            beta = np.linalg.pinv(xmt_wm @ xm) @ xmt_wm @ ym
            y = (beta @ xp)[0]
        else:
            xx = self.n_xx[min_range]
            yy = self.n_yy[min_range]
            sum_weight = np.sum(weights)
            sum_weight_x = np.dot(xx, weights)
            sum_weight_y = np.dot(yy, weights)
            sum_weight_x2 = np.dot(np.multiply(xx, xx), weights)
            sum_weight_xy = np.dot(np.multiply(xx, yy), weights)

            mean_x = sum_weight_x / sum_weight
            mean_y = sum_weight_y / sum_weight

            b = (sum_weight_xy - mean_x * mean_y * sum_weight) / \
                (sum_weight_x2 - mean_x * mean_x * sum_weight)
            a = mean_y - b * mean_x
            y = a + b * n_x
        return self.denormalize_y(y)



In [ ]:
wls=[]
sps=[]
for i in range(0, len(current_experiment.list_of_signals)):
    wavelengths = np.concatenate([w for w in current_experiment.list_of_signals[i].wavelengths])
    spectrum = np.concatenate([s for s in current_experiment.list_of_signals[i].spectrum ])
    wls=np.concatenate([wls,wavelengths])
    sps = np.concatenate([sps,spectrum])

    
xx = wls
yy = sps

loess = Loess(xx, yy)

for x in xx:
    y = loess.estimate(wls, window=5, use_matrix=False, degree=1)
    print(x, y)

In [ ]:
subplots()
plot(xx,yy,'o',markersize=1)
plot(xx,yy,'o',markersize=1)

In [ ]:
for i in range(0, len(sg.wavelengths)):
    plot(sg.wavelengths[i],sg.spectrum[i],'-',color='k')

In [ ]:
from sklearn.neighbors import RadiusNeighborsRegressor
neigh = RadiusNeighborsRegressor(radius=.1)
XX = xx.reshape(-1, 1)
neigh.fit(XX, yy)

In [ ]:
plot(xx,neigh.predict(XX))

In [ ]:
res_wls.fittedvalues

In [ ]:
sm.add_constant(xx)

In [ ]:
#Choose the element for computing the relevant lines
al = element('Ca')

Tp = 1.*T_ref

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 2
ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

ion_energies = al.ion_energies

#ion state 1
nlines = 3
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 2
nlines = 2
l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],al,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)


In [ ]:
n_spectrometer = 2

ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 3
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, 
                                                        n_lines = nlines,lower_limit = ll, upper_limit = ul)

#line to normalize the intensity for direct
line_norm1=lines[1]
print("# Spectrometer 1 normalized to " + str(line_norm1))
sg.wavelengths[n_spectrometer] = sg.wavelengths[n_spectrometer]+0.05

sg.compare_and_peaks(sample, spectrometer=n_spectrometer,max_ion_state=2, 
                                 electron_temperature = temperature,electron_density=density,
                                 d_lambda=0.02, use_wavelengths=True,line_normalize=line_norm1,Plotline = True, resolution = 5000)